In [74]:
# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

### gensim 라이브러리에서 GridSearchCV를 이용하는 방법
- sklearn에서 제공하는 GridSearchCV는 모델의 `fit(), transform(), predict()` 함수를 이용하여 최적의 매개변수 값들을 찾는 class
- gensim의 모델들은 해당 함수들이 존재하지 않는다.
</br> <span style='color:#808080'>(gensim 라이브러리에서는 객체가 생성됨과 동시에 데이터를 넣어 학습하므로, `fit()` 함수가 존재하지 않음. 변환과 예측도 다른 이름의 함수를 사용.)</span>
</br>**$\Rightarrow$ class를 새로 생성하여 `fit(), transform(), predict()` 함수를 내장한 객체를 생성**
- pipeline, kfold, gridsearch( )를 이용

In [75]:
# 라이브러리 로드
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import FastText
from konlpy.tag import Komoran
import numpy as np

In [76]:
# Komoran 형태소 분석기를 이용한 토큰화 함수 정의
komoran = Komoran()
def tokenize():
    return lambda x: [t for t in komoran.morphs(x)]
tokenizer = tokenize()

#### FastText의 매개변수
- 토큰화된 2차원 리스트 데이터
- vector_size(좌표축의 개수, 차원의 수)
- window(확인할 주변 단어 개수)
- min_count(최소 등장 횟수)
- sg(확률 계산 방식, 주변 단어를 찾을 것인가 or 중심 단어를 찾을 것인가)
- epoch(반복 학습 횟수)
- min_n(subword의 최소 개수, 글자 수)
- max_n(subword의 최대 개수(글자 수))
- worker(병렬 처리 스레드의 수)

In [ ]:
# FastText를 fit(), transform() 을 내장한 class로 정의
class FastTextVectorizer(BaseEstimator, TransformerMixin):
    # 생성자 함수: class가 생성될 때 최초로 한 번 실행되는 함수 (class 내부에서 사용할 데이터들을 저장하기 위해 사용)
    def __init__(
            self,   # 클래스가 생성되는 위치값이 저장되는 변수
            # FastText에서 사용할 매개변수
                # 토큰화된 데이터 (2차원 리스트)
                # vector_size : 좌표축의 개수(차원의 수)
                # window : 체크할 주변 단어의 개수
                # min_count : 최소 등장 횟수
                # epochs : 반복 학습의 횟수
                # sg : 확률 계산 방식 (주변 단어를 찾을것인가 중심 단어를 찾을것인가)
                # min_n : subword의 최소 개수(글자 수)
                # max_n : subword의 최대 개수(글자 수)
                # worker : 병렬 처리 스레드의 수
            # FastText 객체가 생성되자마자 사용하기 때문에, 객체를 생성할 때 매개변수를 지정해둬야 한다.
            # fit을 할 때 지정할 수도 있지만, 일반적으론 fit(X_train, Y_train) 외에 값을 넣지 않는다.
            tokenizer = None,
            vector_size = 100,
            min_count = 1,
            window = 3,
            sg = 1,
            epochs = 10,
            min_n = 3,
            max_n = 6,
            workers = 1
    ):
        # class 안에 저장공간을 생성하여 데이터 저장
        self.tokenizer = tokenizer
        self.vector_size = vector_size
        self.min_count = min_count
        self.window = window
        self.sg = sg
        self.epochs = epochs
        self.min_n = min_n
        self.max_n = max_n
        self.workers = workers
        # 시드값 고정
        self.seed = 42
    
        # 모델과 단어 사전이 저장될 빈 공간 생성
        self.model = None
        self.voca = None

    # 총 4개의 메서드 생성
    # 토큰화 메서드
    # 보통 토큰화하는 값들은 X, Y 중 X 
    def to_token(self, X):
        # 만약 토큰화 함수가 존재하지 않는다면(None), 공백을 기준으로 문자를 나눠준다.
        if self.tokenizer is None:
            # 여러 개의 문장이 담긴 데이터를 2차원 배열로 split()
            # s: 문장 하나
            # t: 문장 안의 단어
            return [ [t for t in s.split()] for s in X ]
        # 만약 토큰화 함수가 존재한다면
        return [self.tokenizer(s) for s in X]
    
    # 학습 함수 -> fit( 독립 변수 X, 종속 변수 Y )
    def fit( self, X, Y ):
        # class 내부의 to_token() 함수를 호출
        sentences = self.to_token(X)
        # FastText를 이용하여 sentences 데이터들을 학습
        # 클래스를 생성할 때 받아온 값들이 대입됨
        self.model = FastText(
            sentences= sentences,
            vector_size= self.vector_size,
            min_count= self.min_count,
            window= self.window,
            sg= self.sg,
            epochs= self.epochs,
            min_n= self.min_n,
            max_n= self.max_n,
            workers= self.workers,
            seed= self.seed
        )
        # 학습 후, 학습 데이터에서의 단어 사전 생성
        self.voca = set(self.model.wv.key_to_index.keys())      # set: 집합. 중복을 허용하지 않는 type이라 중복 방지를 위해 사용.
        return self
    
    # 1차 class 테스트

    # 문장에서 평균 벡터로 사용할 _doc_vec() 함수를 정의
    def doc_vec(self, tokens):
        # 하나의 문장을 벡터화 (= 단위 벡터의 평균 구하기)
        # self.model -> FastText 모델
        # wv -> 단어들과 단위 벡터들이 모여있는 dict 형태의 데이터
        vecs = [ self.model.wv[w] for w in tokens if w in self.model.wv ]
        if not vecs:
            # vecs가 존재하지 않는 경우 (빈 리스트인 경우) 희소행렬을 되돌려준다.
            v = np.zeros(self.model.vector_size)
        else:
            v = np.mean(vecs, axis= 0)
        return v
    
    # 변형 함수 정의 (transform 함수)
    # 예측 모델은 따로 있으니 predict 필요 없음
    def transform(self, X):
        # 토큰화
        sentences = self.to_token(X)
        # 임베딩
        mat = np.vstack( [ self.doc_vec(tokens) for tokens in sentences ] )     # 출력 결과가 보기 좋음 + 혹시 모를 오류 방지를 위해 list -> numpy stack 형태(array)로 변환
        return mat

In [78]:
# 1차 class 테스트
X = [
    '상품 품질이 아주 좋다', 
    '배송이 너무 느리다', 
    '포장이 잘되어 있고 만족합니다',
    '환불이 오래 걸려서 불안하다',
    '가격 대비 만족스럽다', 
    '연락이 잘 안 되고 불친절하다', 
    '디자인이 이쁘고 마음에 든다', 
    '설명과 달라서 실망이다', 
    '배송이 빠르고 기사분이 친절했습니다', 
    '색상이 사진과 달라서 만족하지 못했습니다'
]

Y = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]    # 0: 부정, 1: 긍정

In [79]:
# 1차 테스트
vec_data = FastTextVectorizer(tokenizer= tokenizer)

In [80]:
# 2차 테스트
vec_data.fit(X, Y)

,tokenizer,<function tok...002691D715080>
,vector_size,100
,min_count,1
,window,3
,sg,1
,epochs,10
,min_n,3
,max_n,6
,workers,1


In [81]:
# transform 함수 테스트
vec_data.transform(X)
# 여기까지 됐으면 그리드서치, 폴드화 작업 가능

array([[-5.97238715e-04, -7.26660539e-04,  3.44269210e-05,
         1.09036861e-03,  7.23079022e-04,  9.29685135e-04,
        -1.56077824e-03,  2.79494841e-03, -2.58770688e-05,
        -1.57141581e-03,  1.64771534e-03, -7.28481275e-04,
         2.96018575e-03,  4.12621856e-04, -5.18563786e-04,
         8.65457114e-04, -1.97739108e-04, -1.07043155e-03,
         2.28951988e-03, -8.49213509e-04,  9.19712838e-05,
        -1.09151332e-03,  1.20852597e-03,  2.64416449e-03,
         2.42438796e-06, -1.03790918e-03, -3.17062810e-03,
         2.17341562e-03, -2.16587167e-03, -4.07085055e-04,
        -1.30963291e-03, -2.53966398e-04, -1.43039960e-03,
         6.82517653e-04,  1.49218168e-03, -1.73077080e-03,
         7.15601491e-05, -5.09591831e-04, -4.93630650e-05,
         1.15908415e-03,  6.81579579e-04, -1.30134809e-03,
        -3.32375457e-05,  6.79868914e-04,  2.56604369e-04,
         1.84283627e-03, -1.37845473e-03,  1.68437138e-04,
         2.21142778e-03, -5.47966920e-06,  1.08564331e-0

파이프라인 생성
- 로지스틱 회귀를 통해 예측

In [82]:
# FastTextVectorizer 객체와 머신러닝 모델을 합친다.
pipe = Pipeline(
    [
        ('ft', FastTextVectorizer(tokenizer= tokenizer)),
        ('clf', LogisticRegression(max_iter= 2000))
    ]
)

In [83]:
# 이렇게 써도 되지만
pipe.fit(X, Y)

,steps,"[('ft', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,tokenizer,<function tok...002691D715080>
,vector_size,100
,min_count,1
,window,3
,sg,1
,epochs,10
,min_n,3


In [84]:
# 최적의 파라미터를 찾기 위해 GridSearch와 Fold화
cv = StratifiedKFold(n_splits= 5, shuffle= True, random_state= 42)

In [85]:
# 매개변수의 경우들을 dict 데이터로 생성
grid_params = {
    'ft__vector_size': [50, 100],
    'ft__sg': [0, 1],
    'clf__C': [1.0, 2.0]
}

In [86]:
# pipe와 cv, grid_params를 이용하여 GridSearchCV 사용
grid = GridSearchCV(
    estimator= pipe,
    param_grid= grid_params,
    cv= cv,
    scoring= 'f1_macro',
    verbose= 1
)

In [ ]:
# GridSearch 실행
grid.fit(X, Y)
# 최적의 파라미터 
print("GridSearchCV 기준 최적의 파라미터: ", grid.best_params_)
# 최적의 스코어
print("GridSearchCV 기준 최적의 F1-Score: ", grid.best_score_)

# 여기에서 best model을 찾지 않고, 찾아낸 best_params를 이용해 재학습
# 결과가 크게 바뀌지 않을 수도 있지만, 
# 폴드화를 통해 교차검증할 땐 train 데이터를 n등분한 각 폴드마다의 성능을 평균냄.

Fitting 5 folds for each of 8 candidates, totalling 40 fits
GridSearchCV 기준 최적의 파라미터:  {'clf__C': 1.0, 'ft__sg': 0, 'ft__vector_size': 50}
GridSearchCV 기준 최적의 F1-Score:  0.39999999999999997


---
---

#### 연습
**`FastTextVectorizer, StratifiedKfold, Pipeline, GridSearchCV`을 이용하여
</br>data 폴더 안의 `ratings_train.txt` 파일의 데이터를 학습하여 최적의 파라미터를 구하라.**
1. train 데이터 - ratings_train.txt에서 상위 500개의 데이터를 이용
</br> test 데이터 - 하위 100개의 데이터를 이용
2. 파라미터 경우의 수
    - FastText
        - window -> 3, 5
        - sg -> 0, 1
        - epochs -> 10, 50
    - Logistic
        - C -> 1.0, 2.0
3. Fold는 3개를 생성
4. 최적의 파라미터를 생성
5. 최적의 모델을 이용하여 test 데이터를 예측한 뒤 class_report 생성

---
데이터 로드, 전처리

In [88]:
import pandas as pd

In [89]:
# 1. ratings_train.txt 로드 (상위 500개, 하위 100개)
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.head()
# 데이터를 불러오면 결측치, 문자열의 공백, 중복 데이터 제거

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [90]:
# 사용하지 않을 컬럼은 제거 -> id 컬럼
df.drop('id', axis=1, inplace= True)

In [91]:
df.head()

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [92]:
# document 컬럼에서 문자열 좌우의 공백 제거
# 문자열 내장함수인 strip() 이용
df['document'] = df['document'].astype(str)

In [93]:
# Series안의 문자 데이터에서 공백을 제거하는 방법
# 방법 1.
# 하나씩 뽑아서 제거 - for문, map
df['document'].map(lambda x: x.strip())

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 150000, dtype: object

In [94]:
# 방법 2.
# Series에서 String Method를 사용할 수 있는 형태로 변환하고 문자열 함수 사용
df['document'] = df['document'].str.strip()
df['document']

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 150000, dtype: object

In [95]:
# document 컬럼에서 중복 데이터 제거
df.drop_duplicates('document', inplace= True)     # subset: 기준

---

In [96]:
# 훈련 데이터 (상위 500개)와 테스트 데이터 (하위 100개) 분할
X_train = df['document'].head(1000).values
X_test = df['document'].tail(500).values
Y_train = df['label'].head(1000).values
Y_test = df['label'].tail(500).values

In [97]:
# 파이프라인 생성
pipe2 = Pipeline(
    [
        ('ft', FastTextVectorizer(tokenizer= tokenizer)),
        ('clf', LogisticRegression(max_iter= 2000))
    ]
)

In [98]:
cv2 = StratifiedKFold(n_splits= 3, shuffle= True, random_state= 42)

In [99]:
grid_params2 = {
    'ft__window': [3, 5],
    'ft__sg': [0, 1],
    'ft__epochs': [10 ,50],
    'clf__C': [1.0, 2.0]
}

In [100]:
grid2 = GridSearchCV(
    estimator= pipe2,
    cv= cv2,
    param_grid= grid_params2,
    scoring= 'f1_macro',
    verbose= 1
)

In [101]:
grid2.fit(X_train, Y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


,estimator,Pipeline(step..._iter=2000))])
,param_grid,"{'clf__C': [1.0, 2.0], 'ft__epochs': [10, 50], 'ft__sg': [0, 1], 'ft__window': [3, 5]}"
,scoring,'f1_macro'
,n_jobs,None
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,tokenizer,<function tok...002691D715080>


In [102]:
# 최적의 파라미터 
print("GridSearchCV 기준 최적의 파라미터: ", grid2.best_params_)
# 최적의 스코어
print("GridSearchCV 기준 최적의 F1-Score: ", grid2.best_score_)

GridSearchCV 기준 최적의 파라미터:  {'clf__C': 2.0, 'ft__epochs': 50, 'ft__sg': 1, 'ft__window': 3}
GridSearchCV 기준 최적의 F1-Score:  0.7015991575414494


In [103]:
best_model = grid2.best_estimator_

In [104]:
best_pred = best_model.predict(X_test)

In [105]:
print(classification_report(best_pred, Y_test))

              precision    recall  f1-score   support

           0       0.64      0.72      0.67       230
           1       0.73      0.65      0.69       270

    accuracy                           0.68       500
   macro avg       0.68      0.68      0.68       500
weighted avg       0.69      0.68      0.68       500



In [106]:
# 최적의 파라미터를 이용하여 재학습 -> 예측 검증
params = grid2.best_params_

In [107]:
# 값들이 적어 일일히 쳤지만, 아래처럼 pipeline 을 만들어도 됨
pipe3 = Pipeline(
    [
        ('ft', FastTextVectorizer(
            tokenizer= tokenizer,
            window= params['ft__window'],
            sg= params['ft__sg'],
            epochs= params['ft__epochs']
            )),
        ('clf', LogisticRegression(
            max_iter= 2000,
            C= params['clf__C']
        ))
    ]
)

In [108]:
pipe3.fit(X_train, Y_train)

,steps,"[('ft', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,tokenizer,<function tok...002691D715080>
,vector_size,100
,min_count,1
,window,3
,sg,1
,epochs,50
,min_n,3


In [109]:
pred3 = pipe3.predict(X_test)

In [110]:
print(classification_report(pred3, Y_test))
# 데이터의 개수가 적어 차이가 적게 나지만, pipeline을 만들어 수행하는 게 더 효율적

              precision    recall  f1-score   support

           0       0.64      0.72      0.67       230
           1       0.73      0.65      0.69       270

    accuracy                           0.68       500
   macro avg       0.68      0.68      0.68       500
weighted avg       0.69      0.68      0.68       500

